### Data preparation

#### Import packages

In [36]:
!pip install cvxopt


[notice] A new release of pip available: 22.3.1 -> 23.0.1
[notice] To update, run: pip install --upgrade pip


In [37]:
# Install the needed libraries
import pandas as pd
import numpy as np
import cvxopt.modeling
from cvxopt.modeling import variable, op, matrix
import time
import warnings
from pandas.core.common import SettingWithCopyWarning

In [38]:
!pip install openpyxl


[notice] A new release of pip available: 22.3.1 -> 23.0.1
[notice] To update, run: pip install --upgrade pip


In [39]:
# Ignore SettingWithCopyWarning 
warnings.simplefilter(action='ignore', category=SettingWithCopyWarning)

#### Import data

In [40]:
# Import the data
data = pd.read_excel('data_python_modified.xlsx', sheet_name='data', index_col=0)
real = pd.read_excel('data_python_modified.xlsx', sheet_name='реал', index_col=0)
bandwidth = pd.read_excel('data_python_modified.xlsx', sheet_name='ПС', index_col=0)
match = pd.read_excel('data_python_modified.xlsx', sheet_name='match', index_col=0)

#### Prepare data
* First we specify the functions that we use
* Then we apply them to get the final matrixes

Functions that we use for calculations:

#### Calculate the tarif

In [41]:
# Function for calculating the tarif
def calculate_tarif(row):
    if row["Плечо, км"] <= 50:
        return row["Тариф ж/д"] + row["Тариф хранение"] + row["Тариф бренд"] + row["Тариф ВЛ"]
    else:
        return row["Тариф ж/д"] + row["Тариф хранение"] + row["Тариф бренд"] +  row["Тариф ВЛ"] * row["Плечо, км"]

In [42]:
# Function for dividing supply between different brands
def divide_supply(brand_name, match_brand_product):
    
    product_name = match_brand_product[brand_name] 
    sum_brand = real.groupby('НГ')['Объем'].sum().loc[brand_name]
    sum_all = real.groupby('НГ_ПС')['Объем'].sum().loc[product_name]
    p_brand = sum_brand / sum_all
    
    return p_brand

Functions that we use for forming matrixes:

In [43]:
# Function for forming cost matrixes
def form_cost_matrixes(data):
    i = 0
    cost = {}
    for brand in data['НГ'].unique():
        string = "cost_brand_" + str(i)
        cost[string] = data[data['НГ']== brand].pivot_table(index=['НБ'], columns=['ОУ'], values='Тариф', dropna=False)
        i = i + 1
        
    return cost

In [44]:
# Function for forming demand matrixes
def form_demand_matrixes(data, real):
    i = 0
    demand = {}

    for brand in data['НГ'].unique():
        string = "demand_brand_" + str(i) +"_t"
        demand[string] = real[real['НГ']==brand][['ОУ','Объем']].groupby('ОУ').sum().transpose()
        i = i + 1
    
    return demand

In [45]:
# Function for forming supply matrixes
def form_supply_matrixes(data, bandwidth, match_brand_product):
    i = 0
    supply = {}

    for brand in data['НГ'].unique():
    
        product = match_brand_product[brand]
        percent_brand = divide_supply(brand, match_brand_product)
    
        string = "supply_brand_" + str(i)
        supply[string] = bandwidth[bandwidth['НГ_ПС'] == product].groupby('НБ').sum() * percent_brand
        i = i + 1
    
    return supply

In [46]:
# Function for forming final matrixes
def form_final_matrixes(cost, demand, supply):

    i = 0
    matrixes = {}

    for c, d, s in zip(cost.keys(), demand.keys(), supply.keys()):
        trans = pd.concat([cost[str(c)], demand[str(d)]], axis=0)
        string = "matrix_brand_" + str(i)
        matrixes[string] = pd.concat([trans, supply[str(s)]], axis=1)
        i = i + 1
        
    return matrixes

In [47]:
def prepare_data(data, real, bandwidth, match, t):
    # Calculate the tarif
    data['Тариф'] = data.apply(lambda row: calculate_tarif(row), axis=1)
    
    # Subset for the time period
    data = data[data['Дата'] == t]
    real = real[real['Дата'] == t]
    bandwidth = bandwidth[bandwidth['Дата'] == t]
    
    # Adjust datatypes
    data['НБ'] = data['НБ'].map(lambda x: int(x.strip('Нефтебаза ')))
    data['ОУ'] = data['ОУ'].map(lambda x: int(x.strip('АЗС ')))
    real['ОУ'] = real['ОУ'].map(lambda x: int(x.strip('АЗС ')))
    bandwidth['НБ'] = bandwidth['НБ'].map(lambda x: int(x.strip('Нефтебаза ')))
    
    # Replace np.inf with 100 000
    origins_with_inf_supply = list(bandwidth[bandwidth['Объем'] == np.inf]['НБ'].unique())
    bandwidth.loc[:, "Объем"].replace(np.inf, 10**6, inplace=True)
    
    # Create some dictionaries
    match_brand_product = dict(zip(match['НГ'], match['НГ_ПС']))
    match_brand_number = dict(enumerate(match['НГ']))
    
    
    # Create transitionary matrixes
    cost = form_cost_matrixes(data)
    demand = form_demand_matrixes(data, real)
    supply = form_supply_matrixes(data, bandwidth, match_brand_product)
    
    # Create final matrixes
    matrixes = form_final_matrixes(cost, demand, supply)
    
    return matrixes, match_brand_number, origins_with_inf_supply

Applying functions:

In [48]:
# Checking dates
dates = pd.DataFrame({"date" : data['Дата'].unique()})
dates

,date
0,2023-04-01
1,2023-03-01


In [49]:
# Save the first date to the t 
t = dates.iloc[0, 0]

In [50]:
# Get the matrixes
matrixes, match_brand_number, origins_with_inf_supply = prepare_data(data, real, bandwidth, match, t)

In [51]:
# See example:
matrixes['matrix_brand_0']

,0,1,3,4,7,8,10,11,12,15,17,18,19,20,21,22,26,28,43,Объем
0,3268.488947,3241.752289,3232.346002,3245.001733,3205.039266,3331.254535,3217.866021,3237.077649,3238.046782,3285.249240,4259.389025,4439.413324,3159.376018,2935.734418,3000.837326,3228.469471,3200.421634,4756.964716,3264.612417,331.1
1,4808.465100,4808.465100,4808.465100,4808.465100,4808.465100,4808.465100,4808.465100,4808.465100,4808.465100,4808.465100,5402.787462,5595.115521,5424.934992,5210.404717,5272.855284,4808.465100,4808.465100,6504.271376,4808.465100,1000000.0
Объем,29.693803,23.717783,69.617399,17.180032,33.144700,19.220910,42.384663,18.717185,29.952396,8.060745,13.678923,8.212597,4.737933,15.236888,7.962316,14.279678,8.315718,10.245384,15.893195,NaN


In [52]:
match_brand_number

{0: 'Бензин 100 бренд',
 1: 'Бензин 92',
 2: 'Бензин 95',
 3: 'Бензин 95 бренд',
 4: 'Топливо дизельное с присадками летнее'}

In [53]:
origins_with_inf_supply

[1]

### Preparation of data for cvxopt

In [54]:
def overall_data_lists(matrixes):
    result_costs = {}
    overall_costs = [] 
    result_demand = {}
    overall_demand = [] 
    values_n_points_brand = {}
    n_points_overall = 0
    result_supply = {}
    overall_supply = [] 
    
    for i, brand in enumerate(data['НГ'].unique()):
        matrix_brand = matrixes['matrix_brand_{i}'.format(i=i)]
        result_costs["values_costs_{}".format(i)] = np.array(matrix_brand[:-1].iloc[:,:-1].values).flatten().tolist()
        overall_costs += list(result_costs.values())[i]
        result_demand["values_demand_{}".format(i)] = pd.Series(matrix_brand[-1:].iloc[:,:-1].values.tolist())[0]
        overall_demand += list(result_demand.values())[i]
        values_n_points_brand["n_points_{}_t".format(i)] = len(matrix_brand.columns) - 1
        n_points_overall += values_n_points_brand['n_points_{i}_t'.format(i=i)]
        values_supply = matrixes['matrix_brand_{i}'.format(i=i)].iloc[:-1].loc[:, 'Объем']
        result_supply["values_supply_{}".format(i)] = values_supply.values.tolist()
        overall_supply += list(result_supply.values())[i]
        
    n_origins_sum = data['НБ'].nunique()
    n_origins_overall = len(overall_supply)
        
    return result_costs, overall_costs, result_demand, overall_demand, values_n_points_brand, n_points_overall, result_supply, overall_supply, n_origins_sum, n_origins_overall

In [55]:
result_costs, overall_costs, result_demand, overall_demand, values_n_points_brand, n_points_overall, result_supply, overall_supply, n_origins_sum, n_origins_overall = overall_data_lists(matrixes)

In [56]:
n_origins_sum

2

### Applying cvxopt to solve the problem

#### Generate a function we want to minimize

In [57]:
from cvxopt.modeling import variable, op, matrix

In [58]:
# Setting variables and costs for the model

x = variable(n_points_overall * n_origins_sum, 'x') #количество НБ, умноженное на количество АЗС. 180 * 2

c = matrix(np.array(overall_costs).reshape(len(x), 1)) # список костов из 360 ячеек. порядок:  100brand + 92 + 95 + 95brand + disel

In [59]:
z = cvxopt.modeling.dot(x, c)

In [60]:
x.__dict__

{'name': 'x', 'value': None, '_size': 360}

#### Generate restrictions on demand

In [61]:
number_of_constraints = 1
initial_n_of_constraints = number_of_constraints
number_of_x = 0
list_of_constraints = {}

for b in range(len(matrixes)):
    matrix_brand = matrixes['matrix_brand_{b}'.format(b=b)]
    for i in range(len(matrix_brand.columns)-1):
        constraint = 'mass' + str(number_of_constraints)
        
        trans = []
            
            
        string = ""
        
        for j in range(n_origins_sum):
            trans.append(x[i+number_of_x+j*(len(matrix_brand.columns)-1)])
            if j == n_origins_sum - 1:
                end = ''
            else:
                end = ' + '
            string += "x[{x}]{y}".format(x = i+number_of_x+j*(len(matrix_brand.columns)-1), y = end)
            
        list_of_constraints[constraint] = (cvxopt.modeling.sum(trans) == overall_demand[number_of_constraints - initial_n_of_constraints])
            
        print("mass{} = (".format(number_of_constraints) + string + " == " + str(overall_demand[number_of_constraints - initial_n_of_constraints]) + ")")
        print("")
        
        number_of_constraints += 1
        
    number_of_x += (len(matrix_brand.columns)-1) * n_origins_sum


mass1 = (x[0] + x[19] == 29.69380349634518)

mass2 = (x[1] + x[20] == 23.71778272032937)

mass3 = (x[2] + x[21] == 69.61739907583812)

mass4 = (x[3] + x[22] == 17.18003246835099)

mass5 = (x[4] + x[23] == 33.14470001081621)

mass6 = (x[5] + x[24] == 19.22090952214534)

mass7 = (x[6] + x[25] == 42.38466312255857)

mass8 = (x[7] + x[26] == 18.71718475620516)

mass9 = (x[8] + x[27] == 29.95239566535984)

mass10 = (x[9] + x[28] == 8.060744781665605)

mass11 = (x[10] + x[29] == 13.67892325874485)

mass12 = (x[11] + x[30] == 8.212596603459344)

mass13 = (x[12] + x[31] == 4.73793339941212)

mass14 = (x[13] + x[32] == 15.23688750856649)

mass15 = (x[14] + x[33] == 7.962316122394742)

mass16 = (x[15] + x[34] == 14.27967784390616)

mass17 = (x[16] + x[35] == 8.315718258401118)

mass18 = (x[17] + x[36] == 10.24538436128756)

mass19 = (x[18] + x[37] == 15.89319452782291)

mass20 = (x[38] + x[81] == 304.5709837527155)

mass21 = (x[39] + x[82] == 240.3026994876928)

mass22 = (x[40] + x[83] == 193.42

In [62]:
len(list_of_constraints)

180

#### Generate restrictions on supply

In [63]:
# Define restrictions on supply <= upper_const, supply >= lower_const    

# Percents of loading demanded. For now only try a first variable
percents_loading = np.arange(0.5, 1, 0.1)

number_of_x = 0
count = 0

for i, brand in enumerate(data['НГ'].unique()):
        supply_brand_i = matrixes['matrix_brand_{i}'.format(i=i)].iloc[:-1].loc[:, "Объем"]
        values_n_points_i = values_n_points_brand['n_points_{i}_t'.format(i=i)]
        
        for origin in range(n_origins_sum):
            
            #constraint = 'mass' + str(number_of_constraints)
            trans = []
            
            N = number_of_x + values_n_points_i
            
            if origin not in origins_with_inf_supply:
                string = ""

                for j in range(number_of_x, N):
                    trans.append(x[j])
                    if j == N - 1:
                        end = ''
                    else:
                        end = ' + '
                    string += "x[{x}]{y}".format(x = j, y = end)

                constraint = 'mass' + str(number_of_constraints)
                list_of_constraints[constraint] = (cvxopt.modeling.sum(trans) <= overall_supply[count])

                print("mass{} = (".format(number_of_constraints) + string + ") <= " + str(overall_supply[count]) + ")")
                print("")
                number_of_constraints += 1

                constraint = 'mass' + str(number_of_constraints)
                const = np.min(percents_loading[0] * matrixes['matrix_brand_' + str(i)].iloc[:-1].loc[:, 'Объем'])
                list_of_constraints[constraint] = (cvxopt.modeling.sum(trans) >= const)

                print("mass{} = (".format(number_of_constraints) + string + ") >= " + str(const) + ")")
                print("")

                
                number_of_constraints += 1
            count = count + 1
            number_of_x = number_of_x + values_n_points_i

mass181 = (x[0] + x[1] + x[2] + x[3] + x[4] + x[5] + x[6] + x[7] + x[8] + x[9] + x[10] + x[11] + x[12] + x[13] + x[14] + x[15] + x[16] + x[17] + x[18]) <= 331.1)

mass182 = (x[0] + x[1] + x[2] + x[3] + x[4] + x[5] + x[6] + x[7] + x[8] + x[9] + x[10] + x[11] + x[12] + x[13] + x[14] + x[15] + x[16] + x[17] + x[18]) >= 165.55)

mass183 = (x[38] + x[39] + x[40] + x[41] + x[42] + x[43] + x[44] + x[45] + x[46] + x[47] + x[48] + x[49] + x[50] + x[51] + x[52] + x[53] + x[54] + x[55] + x[56] + x[57] + x[58] + x[59] + x[60] + x[61] + x[62] + x[63] + x[64] + x[65] + x[66] + x[67] + x[68] + x[69] + x[70] + x[71] + x[72] + x[73] + x[74] + x[75] + x[76] + x[77] + x[78] + x[79] + x[80]) <= 4663.7)

mass184 = (x[38] + x[39] + x[40] + x[41] + x[42] + x[43] + x[44] + x[45] + x[46] + x[47] + x[48] + x[49] + x[50] + x[51] + x[52] + x[53] + x[54] + x[55] + x[56] + x[57] + x[58] + x[59] + x[60] + x[61] + x[62] + x[63] + x[64] + x[65] + x[66] + x[67] + x[68] + x[69] + x[70] + x[71] + x[72] + x[73] + x[74] + 

In [64]:
#неотрицательные объемы
x_non_negative = (x >= 0)   

In [65]:
list_of_constraints['x_non_negative'] = x_non_negative

In [66]:
len(list_of_constraints)

191

In [67]:
# Final constraints
c = pd.DataFrame(list_of_constraints.items())
c = list(c[:][1].values)
c

[<scalar equality>,
 <scalar equality>,
 <scalar equality>,
 <scalar equality>,
 <scalar equality>,
 <scalar equality>,
 <scalar equality>,
 <scalar equality>,
 <scalar equality>,
 <scalar equality>,
 <scalar equality>,
 <scalar equality>,
 <scalar equality>,
 <scalar equality>,
 <scalar equality>,
 <scalar equality>,
 <scalar equality>,
 <scalar equality>,
 <scalar equality>,
 <scalar equality>,
 <scalar equality>,
 <scalar equality>,
 <scalar equality>,
 <scalar equality>,
 <scalar equality>,
 <scalar equality>,
 <scalar equality>,
 <scalar equality>,
 <scalar equality>,
 <scalar equality>,
 <scalar equality>,
 <scalar equality>,
 <scalar equality>,
 <scalar equality>,
 <scalar equality>,
 <scalar equality>,
 <scalar equality>,
 <scalar equality>,
 <scalar equality>,
 <scalar equality>,
 <scalar equality>,
 <scalar equality>,
 <scalar equality>,
 <scalar equality>,
 <scalar equality>,
 <scalar equality>,
 <scalar equality>,
 <scalar equality>,
 <scalar equality>,
 <scalar equality>,


### Solving the problem

In [68]:
start = time.time()


problem =op(z, c)

problem.solve(solver='glpk')  

print("Результат Xopt:")
for i in x.value:
    print ("x =", i)

print("Стоимость доставки:")
print(problem.objective.value()[0])

stop = time.time()
print ("Время :")
print(stop - start)

GLPK Simplex Optimizer 5.0
550 rows, 360 columns, 1080 non-zeros
      0: obj =   0.000000000e+00 inf =   2.951e+04 (185)
    190: obj =   8.542305209e+07 inf =   0.000e+00 (0)
*   365: obj =   8.406502946e+07 inf =   0.000e+00 (0)
OPTIMAL LP SOLUTION FOUND
Результат Xopt:
x = 19.71473015875068
x = 23.71778272032937
x = 69.61739907583812
x = 17.18003246835099
x = 33.14470001081625
x = -7.105427357601002e-15
x = 42.38466312255857
x = 18.71718475620516
x = 29.95239566535984
x = -1.4210854715202004e-14
x = -2.3092638912203256e-14
x = -1.5987211554602254e-14
x = 4.73793339941212
x = 15.236887508566511
x = 7.962316122394742
x = 14.279677843906171
x = 8.315718258401091
x = 10.245384361287563
x = 15.89319452782291
x = 9.9790733375945
x = 0.0
x = 0.0
x = 0.0
x = 0.0
x = 19.220909522145348
x = 0.0
x = 0.0
x = 0.0
x = 8.060744781665619
x = 13.678923258744874
x = 8.21259660345936
x = 0.0
x = -2.1316282072803006e-14
x = 0.0
x = -1.0658141036401503e-14
x = 2.6645352591003757e-14
x = -3.552713678800

### Postprocessing

In [69]:
# Defining the final products' volumes
def values_costs_for_brand(matrixes):
    volumes_result = {}
    i = 0
    n_brands = data['НГ'].nunique()
    n_origins = max(data['НБ'].str.strip('Нефтебаза ').astype('int'))
    n_points = max(data['ОУ'].str.strip('АЗС ').astype('int'))
    volumes = list(x.value)
    for b in np.arange(0, n_brands):
        matrix_b = matrixes['matrix_brand_{b}'.format(b=b)]
        for o in np.arange(0, n_origins+1):
            for p in np.arange(0, n_points+1):

                if (o in matrix_b.index) and (p in matrix_b.columns):
                    name_of_product = "P_" + str(p) + "_O_"+ str(o) + "_brand_"+ str(b)
                    volumes_result[name_of_product] = volumes[i]
                    i += 1 
    volumes_result_data = pd.DataFrame(volumes_result.items(), columns=['Product', 'Value'])
    return volumes_result_data

volumes_result_data = values_costs_for_brand(matrixes)
volumes_result_data

,Product,Value
0,P_0_O_0_brand_0,19.714730
1,P_1_O_0_brand_0,23.717783
2,P_3_O_0_brand_0,69.617399
3,P_4_O_0_brand_0,17.180032
4,P_7_O_0_brand_0,33.144700
...,...,...
355,P_44_O_1_brand_4,0.000000
356,P_45_O_1_brand_4,0.000000
357,P_46_O_1_brand_4,0.000000
358,P_47_O_1_brand_4,0.000000
